<a href="https://colab.research.google.com/github/Sudeeppp-Mishra/LeafLens/blob/main/LeafLens_Train_on_DataSets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeafLens: AI-Based Plant Disease Detection

LeafLens classifies plant leaf images into healthy or diseased categories.
We use a pretrained ResNet18 model (transfer learning) for accuracy and faster training.

**Libraries used:**
- PIL / OpenCV / NumPy: Image loading and preprocessing
- PyTorch / Torchvision: Model training and evaluation
- Matplotlib: Visualization of metrics
- PySide6 / pyttsx3 / gTTS: GUI + audio output (later integration)

**Dataset:** PlantVillage (Image Classification)  
**Platform:** Google Colab (GPU)

## Why GPU?

Deep learning models require heavy computation.  
Using GPU in Colab speeds up training significantly.

## Mount Google Drive

Accessing PlantVillage dataset stored in Google Drive.  
This avoids repeated uploads and keeps large datasets organized.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Required Libraries

Purpose

We import all required libraries for:

	•	Deep learning (PyTorch)
	•	Image processing (OpenCV, PIL)
	•	Dataset handling
	•	Visualization

We use:

	•	OpenCV → fast, robust image reading
	•	PIL → transformations compatibility
	•	PyTorch → CNN training framework

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
from sklearn.model_selection import train_test_split

## Check GPU

We confirm if GPU is available.  
Training on GPU is much faster than CPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Dataset Path

Why?

We tell Python where PlantVillage dataset is located.

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/datasets/PlantVillage"
print("Classes found:", os.listdir(DATASET_DIR))

Classes found: ['Tomato_healthy', 'PlantVillage', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Leaf_Mold', 'Potato___Early_blight', 'Tomato_Septoria_leaf_spot', 'Potato___Late_blight', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Tomato__Tomato_mosaic_virus', 'Tomato_Bacterial_spot', 'Potato___healthy', 'Pepper__bell___Bacterial_spot', 'Tomato_Late_blight']


## Load Image Paths and Labels

Why??

CNN does not understand folders.

We convert:

	•	Folder names → class labels
	•	Images → file paths

In [ ]:
image_paths = []
labels = []
class_names = sorted(os.listdir(DATASET_DIR))

class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

for cls in class_names:
    cls_path = os.path.join(DATASET_DIR, cls)
    if not os.path.isdir(cls_path):
        continue
    for img in os.listdir(cls_path):
        image_paths.append(os.path.join(cls_path, img))
        labels.append(class_to_idx[cls])

print("Total images:", len(image_paths))
print("Total classes:", len(class_names))

Total images: 20654
Total classes: 16


## Training-Validation Split (80-20)

WHY THIS IS MANDATORY

Why not train on 100% data?

Because:
	•	Model may memorize images (overfitting)
	•	You cannot measure real performance
	•	Teacher will ask: “How do you know it works on unseen data?”

80% → learning

20% → evaluation (unseen images)

In [ ]:
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

print("Training images:", len(train_paths))
print("Validation images:", len(val_paths))

Training images: 16523
Validation images: 4131


## Image Preprocessing (OpenCV + PIL)

Why preprocessing BEFORE training

CNN expects:

	•	Same image size
	•	Normalized pixel values
	•	Clean data

We use:

	•	OpenCV → read image
	•	PIL → apply transforms
	•	Normalization → faster convergence


In [ ]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## SAFE Dataset Class

WHY THIS IS CRITICAL

	•	Some images may be corrupted
	•	OpenCV returns None
	•	This prevents infinite freeze

In [ ]:
class SafePlantDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]

        try:
            image = cv2.imread(path)
            if image is None:
                raise ValueError("Corrupted image")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if self.transform:
                image = self.transform(image)

            return image, self.labels[idx]

        except:
            return self.__getitem__((idx + 1) % len(self.paths))

## DataLoader

- Loads batches efficiently  
- Shuffles training data for better convergence

In [ ]:
train_dataset = SafePlantDataset(train_paths, train_labels, train_transform)
val_dataset = SafePlantDataset(val_paths, val_labels, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

## Model Selection - EfficientNet
Why EfficientNet?

	•	Better accuracy than ResNet
	•	Fewer parameters
	•	Faster on limited GPU

In [ ]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Loss Function & Optimizer

- CrossEntropyLoss for multi-class classification  
- Adam optimizer adapts learning rate for faster convergence

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Model Training

- Forward pass → Compute loss → Backpropagate → Update weights  
- Track training and validation accuracy  
- GPU is used if available for faster computation

Installing tqdm to see progress of training

In [ ]:
!pip install tqdm

In [ ]:
import torch
from tqdm import tqdm
import os

# Parameters
epochs = 5
checkpoint_dir = "/content/drive/MyDrive/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
save_every_n_batches = 100  # adjust for speed vs safety

# OPTIONAL: resume from checkpoint
start_epoch = 0
start_batch = 0
checkpoint_path = None  # set path if resuming

if checkpoint_path and os.path.exists(checkpoint_path):
    print(f"Resuming training from checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    start_batch = checkpoint['batch'] + 1
    print(f"Resuming from epoch {start_epoch+1}, batch {start_batch}")

# Training loop
for epoch in range(start_epoch, epochs):
    model.train()
    running_loss = 0.0

    print(f"\nEpoch {epoch+1}/{epochs}")

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training", leave=False)

    for i, (images, labels) in progress_bar:
        # Skip batches if resuming
        if epoch == start_epoch and i < start_batch:
            continue

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

        # Save checkpoint every N batches
        if (i + 1) % save_every_n_batches == 0:
            checkpoint_file = os.path.join(checkpoint_dir, f"checkpoint_epoch{epoch+1}_batch{i+1}.pth")
            torch.save({
                'epoch': epoch,
                'batch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, checkpoint_file)
            print(f"\nCheckpoint saved: {checkpoint_file}")

    avg_loss = running_loss / len(train_loader)
    print(f"Training Loss Epoch {epoch+1}: {avg_loss:.4f}")

    # Optional: save checkpoint at end of each epoch
    epoch_checkpoint_file = os.path.join(checkpoint_dir, f"checkpoint_epoch{epoch+1}_end.pth")
    torch.save({
        'epoch': epoch,
        'batch': i,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, epoch_checkpoint_file)
    print(f"Epoch {epoch+1} checkpoint saved: {epoch_checkpoint_file}")


Epoch 1/5


Training:  19%|█▉        | 100/517 [20:44<1:23:07, 11.96s/it, loss=0.734]


Checkpoint saved: /content/drive/MyDrive/checkpoints/checkpoint_epoch1_batch100.pth


Training:  39%|███▊      | 200/517 [41:04<1:04:06, 12.14s/it, loss=0.334]


Checkpoint saved: /content/drive/MyDrive/checkpoints/checkpoint_epoch1_batch200.pth


...

Training:  58%|█████▊    | 300/517 [02:01<01:40,  2.17it/s, loss=0.00345]


Checkpoint saved: /content/drive/MyDrive/checkpoints/checkpoint_epoch4_batch300.pth


Training:  77%|███████▋  | 400/517 [02:42<00:53,  2.17it/s, loss=0.00271]


Checkpoint saved: /content/drive/MyDrive/checkpoints/checkpoint_epoch4_batch400.pth


Training:  80%|███████▉  | 412/517 [02:47<00:42,  2.45it/s, loss=0.00696]